In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)

from modules.import_data import import_data_standard
from modules.tools import return_device

In [ ]:
# hyperparams
learning_rate = 0.0004
batch_size = 32
num_epochs = 64
dropout_rate = 0.55

device = return_device()
target = 'icd10h_category'

Using device: cuda


In [21]:
df, _ = import_data_standard(target=target)

In [23]:
deathcauses,labels = df["tidy_cod"].tolist(), df[target].tolist()

In [10]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", use_fast=True)

config = AutoConfig.from_pretrained(
    "meta-llama/Llama-3.2-1B",
    num_labels=output_dim,           
    classifier_dropout=dropout_rate, 
)

model = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Llama-3.2-1B",
    config=config
).to(device)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class DatasetClass(Dataset):
    def __init__(self, texts, labels):
        self.X  = texts
        self.y = labels
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        return {
            "X":   self.X[idx],
            "y":  self.y[idx]
        }

def collate_fn(batch):
    X  = [ex["X"] for ex in batch]
    y = torch.tensor([ex["y"] for ex in batch], dtype=torch.long)
    enc = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,
    )
    enc["labels"] = y
    return enc